## Bull Call Spread Options Tool

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# # Get Live Stock Price
# from alpha_vantage.timeseries import TimeSeries
# api_key = 'KL1DCLY4P13SDVGR'
# ts = TimeSeries(key=api_key, output_format='pandas')# Initialize TimeSeries object
# data, meta_data = ts.get_quote_endpoint(symbol='TGT')# Retrieve intraday data
#stockPrice = float(data['05. price'][0])

# Inputs
stockPrice = 125.24
days_til_expiration = 41
risk_free_rate = 0.0441
data = [
    {"lowPrice": 2.24, "lowStrike": 130, "highPrice": 0.15, "highStrike": 160, "IV1": 0.24, "IV2": 0.3675},
    {"lowPrice": 3.20, "lowStrike": 125, "highPrice": 0.27, "highStrike": 160, "IV1": 0.40, "IV2": 0.60},
    {"lowPrice": 3.20, "lowStrike": 125, "highPrice": 0.31, "highStrike": 145, "IV1": 0.40, "IV2": 0.60},
    {"lowPrice": 1.80, "lowStrike": 130, "highPrice": 0.31, "highStrike": 145, "IV1": 0.40, "IV2": 0.60},
    {"lowPrice": 3.20, "lowStrike": 125, "highPrice": 0.55, "highStrike": 140, "IV1": 0.40, "IV2": 0.60}
]
expiration_data = [days_til_expiration/365] * len(data)

# Compute Weighted Implied Volatility (IV)
volatility_data = []
for spread in data:
    weighted_iv = (spread["IV1"] * spread["lowPrice"] + spread["IV2"] * spread["highPrice"]) / (spread["lowPrice"] + spread["highPrice"])
    volatility_data.append(weighted_iv)


# Breakeven POP Function
def calculate_breakevenPOP(stockPrice, breakeven, volatility, time_to_expiration):
    d1_breakeven = (np.log(stockPrice / breakeven) + (risk_free_rate + 0.5 * volatility**2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
    breakevenPOP = 1 - stats.norm.cdf(d1_breakeven)
    return breakevenPOP

# Low POP Function
def calcualte_lowPOP(stockPrice, lowStrike, volatility, time_to_expiration):
    d1_low = (np.log(stockPrice / lowStrike) + (risk_free_rate + 0.5 * volatility**2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
    d2_low = d1_low - volatility * np.sqrt(time_to_expiration)
    lowPOP = 1 - stats.norm.cdf(d1_low)
    return d1_low, lowPOP

# Low POP Function
def calcualte_highPOP(stockPrice, highStrike, volatility, time_to_expiration):
    d1_high = (np.log(stockPrice / highStrike) + (risk_free_rate + 0.5 * volatility**2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
    d2_high = d1_high - volatility * np.sqrt(time_to_expiration)
    highPOP = 1 - stats.norm.cdf(d1_high)
    return d1_high, highPOP


# d1_low Black-Scholes Function
def calculate_lowBlackScholes(d1_low,d2_low,risk_free_rate,time_to_expiration,lowStrike,stockPrice):
  lowBlackScholes= (stats.norm.cdf(d1_low) * stockPrice) - (stats.norm.cdf(d2_low)*lowStrike*np.exp(-risk_free_rate*time_to_expiration))
  return lowBlackScholes

# d1_high Black-Scholes Function
def calculate_highBlackScholes(d1_high,d2_high,risk_free_rate,time_to_expiration,highStrike,stockPrice):
  highBlackScholes= (stats.norm.cdf(d1_high) * stockPrice) - (stats.norm.cdf(d2_high)*highStrike*np.exp(-risk_free_rate*time_to_expiration))
  return highBlackScholes

# Main Function
def calculate_main(lowPrice, lowStrike, highPrice, highStrike, volatility, time_to_expiration):
    netPremium = lowPrice - highPrice
    breakeven = lowStrike + netPremium
    delta = highStrike - lowStrike
    maxProfit = (delta - netPremium) * 100
    maxLoss = netPremium * 100
    rtr = maxLoss / maxProfit # Risk to Reward
    breakevenPOP = calculate_breakevenPOP(stockPrice, breakeven, volatility, time_to_expiration)

    d1_low, lowPOP = calcualte_lowPOP(stockPrice, lowStrike, volatility, time_to_expiration)
    d2_low = d1_low - volatility * np.sqrt(time_to_expiration)
    d1_high, highPOP = calcualte_highPOP(stockPrice, highStrike, volatility, time_to_expiration)
    d2_high = d1_high - volatility * np.sqrt(time_to_expiration)

    #lowPOP = calcualte_lowPOP(stockPrice, lowStrike, volatility, time_to_expiration)
    #highPOP = calcualte_highPOP(stockPrice, highStrike, volatility, time_to_expiration)
    bpm= ((breakeven-stockPrice)/stockPrice)*100 #breakeven percentage move
    leverage = maxProfit/maxLoss

    lowBlackScholes = calculate_lowBlackScholes(d1_low, d2_low, risk_free_rate, time_to_expiration, lowStrike, stockPrice)
    highBlackScholes = calculate_highBlackScholes(d1_high, d2_high, risk_free_rate, time_to_expiration, highStrike, stockPrice)
    return {
        "Spread": f"{lowStrike}-{highStrike}",
        "Breakeven": round(breakeven, 2),
        "Breakeven % Move": round(bpm, 2),
        #"Net Premium": round(netPremium, 2),
        "Leverage": round(leverage, 4),
        "Max Loss": round(maxLoss, 2),
        "Max Profit": round(maxProfit, 2),
        "Risk-to-Reward": round(rtr, 4),
        "LOW POP": round(lowPOP, 4),
        "Breakeven POP": round(breakevenPOP, 4),
        "High POP": round(highPOP, 4),
        "Low Strike Price": round(lowPrice,2),
        "Low Strike Black-Scholes": round(lowBlackScholes, 2),
        "High Strike Price": round(highPrice,2),
        "High Strike Black-Scholes": round(highBlackScholes, 2)

    }

# Call calculate_main function
results = []
for i, spread in enumerate(data):
    volatility = volatility_data[i]
    time_to_expiration = expiration_data[i]

    results.append(calculate_main(
        spread["lowPrice"], spread["lowStrike"],
        spread["highPrice"], spread["highStrike"],
        volatility, time_to_expiration
    ))

# Display Output
df = pd.DataFrame(results)
display(df)
print("Stock Price:", stockPrice)

,Spread,Breakeven,Breakeven % Move,Leverage,Max Loss,Max Profit,Risk-to-Reward,LOW POP,Breakeven POP,High POP,Low Strike Price,Low Strike Black-Scholes,High Strike Price,High Strike Black-Scholes
0,130-160,132.09,5.47,13.3541,209.0,2791.0,0.0749,0.6359,0.7052,0.9978,2.24,2.48,0.15,0.01
1,125-160,127.93,2.15,10.9454,293.0,3207.0,0.0914,0.4526,0.5189,0.9509,3.20,7.37,0.27,0.34
2,125-145,127.89,2.12,5.9204,289.0,1711.0,0.1689,0.4526,0.5176,0.8267,3.20,7.40,0.31,1.53
3,130-145,131.49,4.99,9.0671,149.0,1351.0,0.1103,0.5607,0.5917,0.8190,1.80,5.43,0.31,1.66
4,125-140,127.65,1.92,4.6604,265.0,1235.0,0.2146,0.4524,0.5104,0.7479,3.20,7.60,0.55,2.53


Stock Price: 125.24
